<a href="https://colab.research.google.com/github/AnumandlaS/AI-or-Human/blob/main/Just_testing_macformer_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas numpy scikit-learn tensorflow torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

In [ ]:
!pip install keras

In [15]:
import pandas as pd
from transformers import AutoTokenizer
import torch
from keras.preprocessing.sequence import pad_sequences

def load_data(filepath):
    df = pd.read_csv(filepath)
    return df['source'].tolist(), df['target'].tolist()

def tokenize_with_chemberta(tokenizer, sequences):
    """
    Tokenizes SMILES strings with ChemBERTa and adds start (<s>) and end (</s>) tokens.
    """
    return [["<s>"] + tokenizer.tokenize(seq) + ["</s>"] for seq in sequences]

def encode_sequences_with_chemberta(tokenizer, sequences):
    """
    Converts tokenized SMILES into IDs using ChemBERTa vocabulary.
    """
    return [tokenizer.convert_tokens_to_ids(seq) for seq in sequences]

# def pad_sequences_to_fixed_length(sequences, max_length):
#     """
#     Pads tokenized sequences to a fixed length using 0 for padding.
#     """
#     return pad_sequences(sequences, maxlen=max_length, padding='post', value=0)

def pad_sequences_to_fixed_length(sequences, max_length, pad_token_id):
    return pad_sequences(sequences, maxlen=max_length, padding='post', value=pad_token_id)

def preprocess_data(filepath, max_length=200):
    tokenizer = AutoTokenizer.from_pretrained("DeepChem/ChemBERTa-77M-MLM")
    src, tgt = load_data(filepath)
    src_tokenized = tokenize_with_chemberta(tokenizer, src)
    tgt_tokenized = tokenize_with_chemberta(tokenizer, tgt)
    src_encoded = encode_sequences_with_chemberta(tokenizer, src_tokenized)
    tgt_encoded = encode_sequences_with_chemberta(tokenizer, tgt_tokenized)
    vocab = tokenizer.get_vocab()
    pad_token = '<pad>' if '<pad>' in vocab else '[PAD]'
    if pad_token not in vocab:
        tokenizer.add_tokens([pad_token])
        vocab = tokenizer.get_vocab()
    pad_token_id = vocab[pad_token]
    src_padded = pad_sequences_to_fixed_length(src_encoded, max_length, pad_token_id)
    tgt_padded = pad_sequences_to_fixed_length(tgt_encoded, max_length, pad_token_id)
    return src_padded, tgt_padded, vocab, vocab, pad_token_id

In [17]:
import torch
from torch.utils.data import Dataset
import pandas as pd

class CustomDataset(Dataset):
    # def __init__(self, src_data, tgt_data, src_vocab, tgt_vocab):
    #     self.src_data = src_data
    #     self.tgt_data = tgt_data
    #     self.src_vocab = src_vocab
    #     self.tgt_vocab = tgt_vocab
    #     self.pad_idx = src_vocab.get('<pad>', 0)
    def __init__(self, src_data, tgt_data, src_vocab, tgt_vocab, pad_idx):
        self.src_data = src_data
        self.tgt_data = tgt_data
        self.src_vocab = src_vocab
        self.tgt_vocab = tgt_vocab
        self.pad_idx = pad_idx


    def __len__(self):
        return len(self.src_data)

    def __getitem__(self, idx):
        src = self.src_data[idx]
        tgt = self.tgt_data[idx]
        # Convert to Long (int64) tensor type
        return torch.tensor(src, dtype=torch.long), torch.tensor(tgt, dtype=torch.long)

In [5]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# class PositionalEncoding(nn.Module):
#     def __init__(self, d_model, dropout=0.1, max_len=5000):
#         super(PositionalEncoding, self).__init__()
#         self.dropout = nn.Dropout(p=dropout)

#         pe = torch.zeros(max_len, d_model)
#         position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
#         div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(torch.log(torch.tensor(10000.0)) / d_model))
#         pe[:, 0::2] = torch.sin(position * div_term)
#         pe[:, 1::2] = torch.cos(position * div_term)
#         pe = pe.unsqueeze(0)
#         self.register_buffer('pe', pe)

#     def forward(self, x):
#         x = x + self.pe[:, :x.size(1)]
#         return self.dropout(x)

# class EncoderLayer(nn.Module):
#     def __init__(self, d_model, nhead, dim_feedforward, dropout=0.1):
#         super(EncoderLayer, self).__init__()
#         self.self_attn = nn.MultiheadAttention(d_model, nhead)
#         self.linear1 = nn.Linear(d_model, dim_feedforward)
#         self.dropout1 = nn.Dropout(dropout)
#         self.linear2 = nn.Linear(dim_feedforward, d_model)
#         self.dropout2 = nn.Dropout(dropout)
#         self.norm1 = nn.LayerNorm(d_model)
#         self.norm2 = nn.LayerNorm(d_model)

#     def forward(self, src):
#         src = src.transpose(0, 1)  # [batch_size, seq_len, d_model] -> [seq_len, batch_size, d_model]
#         src2 = self.norm1(src)
#         src2, _ = self.self_attn(src2, src2, src2)
#         src = src + self.dropout1(src2)
#         src2 = self.norm2(src)
#         src2 = self.linear2(self.dropout2(F.relu(self.linear1(src2))))
#         src = src + src2
#         return src.transpose(0, 1)  # Back to [batch_size, seq_len, d_model]

# class DecoderLayer(nn.Module):
#     def __init__(self, d_model, nhead, dim_feedforward, dropout=0.1):
#         super(DecoderLayer, self).__init__()
#         self.self_attn = nn.MultiheadAttention(d_model, nhead)
#         self.cross_attn = nn.MultiheadAttention(d_model, nhead)
#         self.linear1 = nn.Linear(d_model, dim_feedforward)
#         self.dropout1 = nn.Dropout(dropout)
#         self.linear2 = nn.Linear(dim_feedforward, d_model)
#         self.dropout2 = nn.Dropout(dropout)
#         self.norm1 = nn.LayerNorm(d_model)
#         self.norm2 = nn.LayerNorm(d_model)
#         self.norm3 = nn.LayerNorm(d_model)

#     def forward(self, tgt, memory, tgt_mask=None):
#         tgt = tgt.transpose(0, 1)  # [batch_size, seq_len, d_model] -> [seq_len, batch_size, d_model]
#         memory = memory.transpose(0, 1)
#         tgt2 = self.norm1(tgt)
#         tgt2, _ = self.self_attn(tgt2, tgt2, tgt2, attn_mask=tgt_mask)
#         tgt = tgt + self.dropout1(tgt2)
#         tgt2 = self.norm2(tgt)
#         tgt2, _ = self.cross_attn(tgt2, memory, memory)
#         tgt = tgt + self.dropout2(tgt2)
#         tgt2 = self.norm3(tgt)
#         tgt2 = self.linear2(self.dropout2(F.relu(self.linear1(tgt2))))
#         tgt = tgt + tgt2
#         return tgt.transpose(0, 1)  # Back to [batch_size, seq_len, d_model]

# class MacFormer(nn.Module):
#     def __init__(self, src_vocab_size, tgt_vocab_size, start_token=1, eos_token=2, d_model=256, nhead=4, num_encoder_layers=4, num_decoder_layers=4, dim_feedforward=512, dropout=0.1):
#         super(MacFormer, self).__init__()
#         self.src_embedding = nn.Embedding(src_vocab_size, d_model)
#         self.tgt_embedding = nn.Embedding(tgt_vocab_size, d_model)
#         self.positional_encoding = PositionalEncoding(d_model, dropout)
#         self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, nhead, dim_feedforward, dropout) for _ in range(num_encoder_layers)])
#         self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, nhead, dim_feedforward, dropout) for _ in range(num_decoder_layers)])
#         self.fc_out = nn.Linear(d_model, tgt_vocab_size)
#         self.start_token = start_token  # Added for compatibility
#         self.eos_token = eos_token      # Added for compatibility

#     def forward(self, src, tgt):
#         src = self.positional_encoding(self.src_embedding(src))
#         tgt = self.positional_encoding(self.tgt_embedding(tgt))

#         for layer in self.encoder_layers:
#             src = layer(src)

#         tgt_mask = self.generate_square_subsequent_mask(tgt.size(1)).to(tgt.device)
#         for layer in self.decoder_layers:
#             tgt = layer(tgt, src, tgt_mask=tgt_mask)

#         return self.fc_out(tgt)

#     def generate_square_subsequent_mask(self, sz):
#         mask = torch.triu(torch.ones(sz, sz), diagonal=1).bool()
#         return mask

# Define the Transformer model
# Simplified Transformer model using nn.Transformer
import torch.nn as nn
class MacFormer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, start_token=1, eos_token=2,
                 d_model=256, nhead=4, num_encoder_layers=4, num_decoder_layers=4,
                 dim_feedforward=512, dropout=0.1):
        super(MacFormer, self).__init__()
        self.src_embedding = nn.Embedding(src_vocab_size, d_model)
        self.tgt_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = nn.Parameter(torch.zeros(1, 200, d_model))  # Fixed max_length=200
        self.transformer = nn.Transformer(
            d_model=d_model,
            nhead=nhead,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True
        )
        self.fc_out = nn.Linear(d_model, tgt_vocab_size)
        self.start_token = start_token
        self.eos_token = eos_token

    def forward(self, src, tgt):
        src_emb = self.src_embedding(src) + self.positional_encoding[:, :src.size(1), :]
        tgt_emb = self.tgt_embedding(tgt) + self.positional_encoding[:, :tgt.size(1), :]

        # Generate target mask for autoregressive property
        tgt_mask = nn.Transformer.generate_square_subsequent_mask(tgt.size(1)).to(tgt.device)

        # Forward pass through transformer
        output = self.transformer(src_emb, tgt_emb, tgt_mask=tgt_mask)
        return self.fc_out(output)

In [22]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# def train_model(model, dataloader, criterion, optimizer,src_vocab, tgt_vocab, epochs=10, device='cpu'):
#     model.train()
#     a=-1
#     for epoch in range(epochs):
#         total_loss = 0
#         for src, tgt in dataloader:
#             src, tgt = src.to(device), tgt.to(device)
#             optimizer.zero_grad()
#             outputs = model(src, tgt)
#             # Ensure target is of type Long
#             loss = criterion(outputs.view(-1, outputs.size(-1)), tgt.view(-1).long())
#             loss.backward()
#             optimizer.step()
#             total_loss += loss.item()
#         avg_loss = total_loss / len(dataloader)
#         if(epoch%10==0):
#             print(f'Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}')
#             print("\nSample Predictions from Test Set:")
#             test_filepath = r'/content/test.csv'
#             test_src_data, test_tgt_data, test_src_vocab, test_tgt_vocab = preprocess_data(test_filepath, 200)
#             test_dataset = CustomDataset(test_src_data, test_tgt_data, src_vocab, tgt_vocab)
#             test_dataloader = DataLoader(test_dataset, batch_size=32)
#             if a>50 :
#               a=0
#             for i in [a+1]:  # Show first three examples
#                 a+=1
#                 src_smile, tgt_smile, pred_smile = predict_smile(model, test_dataset, src_vocab, tgt_vocab, i, device)

#                 print(f"\nExample {i+1}:")
#                 print(f"Source SMILES: {src_smile}",len(src_smile))
#                 print(f"Target SMILES: {tgt_smile}",len(tgt_smile))
#                 print(f"Predic SMILES: {pred_smile}")
#                 print(f"Predicted length: {len(pred_smile)}")

def train_model(model, dataloader, criterion, optimizer, epochs=10, device='cpu'):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for src, tgt in dataloader:
            src, tgt = src.to(device), tgt.to(device)
            optimizer.zero_grad()
            outputs = model(src, tgt[:, :-1])
            loss = criterion(outputs.reshape(-1, outputs.size(-1)), tgt[:, 1:].reshape(-1).long())
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(dataloader)
        # if epoch % 10 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}')

def evaluate_model(model, dataloader, criterion, device, pad_idx):
    model.eval()
    total_loss = 0
    total_correct = 0
    total_tokens = 0

    with torch.no_grad():
        for src, tgt in dataloader:
            src, tgt = src.to(device), tgt.to(device)
            outputs = model(src, tgt)

            loss = criterion(outputs.view(-1, outputs.size(-1)), tgt.view(-1).long())
            total_loss += loss.item()

            # Calculate accuracy
            predictions = outputs.argmax(dim=-1)
            mask = (tgt != pad_idx)
            correct = (predictions == tgt) & mask
            total_correct += correct.sum().item()
            total_tokens += mask.sum().item()

    avg_loss = total_loss / len(dataloader)
    accuracy = total_correct / total_tokens if total_tokens > 0 else 0

    return avg_loss, accuracy


# def predict_smile(model, dataset, src_vocab, tgt_vocab, index, device):
#     model.eval()
#     with torch.no_grad():
#         # Get source and target from dataset
#         src, tgt = dataset[index]
#         src = src.unsqueeze(0).to(device)  # Add batch dimension
#         tgt = tgt.unsqueeze(0).to(device)  # Add batch dimension

#         # Get model prediction
#         outputs = model(src, tgt)  # Using actual target during prediction
#         predicted = outputs.argmax(dim=-1)

#         # Convert indices back to tokens (without padding)

#         src_tokens = [list(src_vocab.keys())[list(src_vocab.values()).index(idx.item())]
#                      for idx in src[0] if idx.item() not in [0]]
#         tgt_tokens = [list(tgt_vocab.keys())[list(tgt_vocab.values()).index(idx.item())]
#                      for idx in tgt[0] if idx.item() not in [0]]
#         pred_tokens = [list(tgt_vocab.keys())[list(tgt_vocab.values()).index(idx.item())]
#                      for idx in predicted[0] if idx.item() not in [0]]
#         pred_tokens =[]
#         for idx in predicted[0]:
#           if idx.item() in [tgt_vocab['</s>']]:
#             pred_tokens.append(list(tgt_vocab.keys())[list(tgt_vocab.values()).index(idx.item())])
#             break
#           if idx.item() not in [0]:
#             pred_tokens.append(list(tgt_vocab.keys())[list(tgt_vocab.values()).index(idx.item())])

#         # Join tokens without spaces for SMILES format
#         return ' '.join(src_tokens), ' '.join(tgt_tokens), ' '.join(pred_tokens)

# New function to predict macrocyclic SMILES from an acyclic SMILES string
def predict_macrocyclic_smiles(model, smiles_string, tokenizer, vocab, pad_idx, max_length=200, device='cpu'):
    model.eval()
    with torch.no_grad():
        # Tokenize and encode the input SMILES string
        tokenized = ["<s>"] + tokenizer.tokenize(smiles_string) + ["</s>"]
        encoded = tokenizer.convert_tokens_to_ids(tokenized)
        src_padded = pad_sequences_to_fixed_length([encoded], max_length, pad_idx)
        src_tensor = torch.tensor(src_padded, dtype=torch.long).to(device)

        # Initialize target sequence with <s> token
        tgt_start = torch.tensor([[vocab["<s>"]]], dtype=torch.long).to(device)

        # Generate output sequence step-by-step
        generated_ids = [vocab["<s>"]]
        for _ in range(max_length):
            outputs = model(src_tensor, tgt_start)
            next_token_id = outputs[:, -1, :].argmax(dim=-1).item()
            generated_ids.append(next_token_id)
        # Convert token IDs back to SMILES string

            if next_token_id == vocab["</s>"]:
                break

            tgt_start = torch.tensor([generated_ids], dtype=torch.long).to(device)

        pred_tokens = [list(vocab.keys())[list(vocab.values()).index(idx)]
                      for idx in generated_ids if idx != pad_idx]
        pred_smiles = ''.join(pred_tokens).replace("<s>", "").replace("</s>", "")

        return pred_smiles


# def main():
#     # Load and preprocess training data
#     train_filepath = r'/content/train.csv'
#     src_data, tgt_data, src_vocab, tgt_vocab = preprocess_data(train_filepath,200)


#     dataset = CustomDataset(src_data, tgt_data, src_vocab, tgt_vocab)
#     dataloader = DataLoader(dataset, batch_size=32)

#     # Initialize model, criterion, and optimizer
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model = MacFormer(len(src_vocab), len(tgt_vocab)).to(device)
#     criterion = nn.CrossEntropyLoss(ignore_index=0)
#     optimizer = optim.Adam(model.parameters(), lr=0.0001)

#     # Train the model
#     train_model(model, dataloader, criterion, optimizer, src_vocab, tgt_vocab, epochs=1, device=device)

#     # Load and preprocess testing data
#     test_filepath = r'/content/test.csv'
#     test_src_data, test_tgt_data, test_src_vocab, test_tgt_vocab = preprocess_data(test_filepath,200)
#     test_dataset = CustomDataset(test_src_data, test_tgt_data, test_src_vocab, test_tgt_vocab)
#     test_dataloader = DataLoader(test_dataset, batch_size=32)


#     # Evaluate the model
#     test_loss, test_accuracy = evaluate_model(model, test_dataloader, criterion, device,test_dataset.pad_idx)
#     print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

#     print("\nSample Predictions from Test Set:")
#     for i in [0, 1, 2]:  # Show first three examples
#         src_smile, tgt_smile, pred_smile =predict_macrocyclic_smiles(model, test_dataset, test_src_vocab, test_tgt_vocab, i, device)
#         print(f"\nExample {i+1}:")
#         print(f"Source SMILES: {src_smile}")
#         print(f"Target SMILES: {tgt_smile}")
#         print(f"Predicted SMILES: {len(pred_smile)}")

# Main function with example usage
def main():
    # Load and preprocess training data
    train_filepath = r'/content/train.csv'
    src_data, tgt_data, src_vocab, tgt_vocab, pad_idx = preprocess_data(train_filepath, 200)
    dataset = CustomDataset(src_data, tgt_data, src_vocab, tgt_vocab, pad_idx)
    dataloader = DataLoader(dataset, batch_size=32)

    # Initialize model and training setup
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = MacFormer(len(src_vocab), len(tgt_vocab), pad_idx).to(device)
    criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)
    optimizer = optim.Adam(model.parameters(), lr=0.0001)

    # Train the model
    train_model(model, dataloader, criterion, optimizer, epochs=10, device=device)

    # Load tokenizer for prediction
    tokenizer = AutoTokenizer.from_pretrained("DeepChem/ChemBERTa-77M-MLM")
    if '<pad>' not in tokenizer.get_vocab():
        tokenizer.add_tokens(['<pad>'])

    # Example: Predict macrocyclic SMILES from an acyclic SMILES string
    acyclic_smiles = "N 3 * C C C C ( N C ( = O ) C ( C c 1 c n c 2 c c c c c 1 2 ) N C ( = O ) C ( C C C N = C ( N ) N ) N C ( = O ) C ( C c 1 c c c c c 1 ) N C ( = O ) c 1 c c c c ( * ) n 1 ) C ( N ) = O"  # Example acyclic compound (pentane)
    predicted_macrocyclic_smiles = predict_macrocyclic_smiles(
        model, acyclic_smiles, tokenizer, tgt_vocab, pad_idx, device=device
    )
    print(f"\nInput Acyclic SMILES: {acyclic_smiles}")
    print(f"Predicted Macrocyclic SMILES: {predicted_macrocyclic_smiles}")

if __name__ == "__main__":
    main()

#actual target - N C ( = O ) C 1 C C C C N C ( = O ) c 2 c c c c ( n 2 ) C ( = O ) N C ( C c 2 c c c c c 2 ) C ( = O ) N C ( C C C N = C ( N ) N ) C ( = O ) N C ( C c 2 c n c 3 c c c c c 2 3 ) C ( = O ) N 1
#predicted - NC(=O)C1CCCCNC(=O)c2cccc2)C(=O)NC(Cc2ccccc2)C(=O)NCCCCCCCN=C(N)N)C(=O)NC(Cc2cnc3cccc23)C(=O)NC1

Epoch 1/10, Loss: 2.0766
Epoch 2/10, Loss: 1.1472
Epoch 3/10, Loss: 0.9641
Epoch 4/10, Loss: 0.8578
Epoch 5/10, Loss: 0.7683
Epoch 6/10, Loss: 0.6888
Epoch 7/10, Loss: 0.6237
Epoch 8/10, Loss: 0.5668
Epoch 9/10, Loss: 0.5208
Epoch 10/10, Loss: 0.4802

Input Acyclic SMILES: N 3 * C C C C ( N C ( = O ) C ( C c 1 c n c 2 c c c c c 1 2 ) N C ( = O ) C ( C C C N = C ( N ) N ) N C ( = O ) C ( C c 1 c c c c c 1 ) N C ( = O ) c 1 c c c c ( * ) n 1 ) C ( N ) = O
Predicted Macrocyclic SMILES: NC(=O)C1CCCCNC(=O)c2cccc2)C(=O)NC(Cc2ccccc2)C(=O)NCCCCCCCN=C(N)N)C(=O)NC(Cc2cnc3cccc23)C(=O)NC1
